In [1]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import matplotlib.pyplot as plt # 데이터 시각화

from datetime import datetime, timedelta # 시간 데이터 처리
from statsmodels.tsa.arima_model import ARIMA # ARIMA 모델
%matplotlib inline

In [2]:
train = pd.read_csv("./DACON_DATASET/train.csv")
submission = pd.read_csv("./DACON_DATASET/submission_1002.csv")

train['Time'] = pd.to_datetime(train['Time']) 
train = train.set_index('Time')

In [4]:
train.head()

,X692,X1272,X553,X1299,X4,X598,X1003,X1010,X1216,X1047,...,X1164,X656,X1287,X324,X767,X1271,X1288,X58,X1033,X1163
Time,,,,,,,,,,,,,,,,,,,,,
2016-07-26 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-26 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
place_id=[]; time=[] ; target=[] # 빈 리스트를 생성합니다.
for i in train.columns:
    for j in range(len(train)):
        place_id.append(i) # place_id에 미터 ID를 정리합니다.
        time.append(train.index[j]) # time에 시간대를 정리합니다.
        target.append(train[i].iloc[j]) # target에 전력량을 정리합니다.

new_df=pd.DataFrame({'place_id':place_id,'time':time,'target':target})
new_df=new_df.dropna() # 결측치를 제거합니다.
new_df=new_df.set_index('time') # time을 인덱스로 저장합니다.
new_df.head()

,place_id,target
time,,
2018-02-15 00:00:00,X692,0.150
2018-02-15 01:00:00,X692,0.422
2018-02-15 02:00:00,X692,0.314
2018-02-15 03:00:00,X692,0.309
2018-02-15 04:00:00,X692,0.327


In [15]:
import itertools
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
pdq

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [18]:
def get_optimal_params(y):

    param_dict = {}
    for param in pdq:
        try:
            model = ARIMA(y, order=param)
            results_ARIMA = model.fit(disp=-1)
            param_dict[results_ARIMA.aic] = param
        except:
            continue

    min_aic = min(param_dict.keys())
    optimal_params = param_dict[min_aic]
    return optimal_params

In [19]:
agg={}
for key in new_df['place_id'].unique(): # 미터ID 200개의 리스트를 unique()함수를 통해 추출합니다.
    temp = new_df.loc[new_df['place_id']==key] # 미터ID 하나를 할당합니다.
    temp_1h=temp.resample('1h').sum() # 1시간 단위로 정리합니다.
    temp_1day=temp.resample('D').sum() # 1일 단위로 정리합니다.

    # 시간별 예측
    model = ARIMA(temp_1h['target'], order=get_optimal_params(temp_1h['target'])) # AIC를 최소화하는 최적의 파라미터로 모델링합니다.
    results_ARIMA = model.fit(disp=-1)
    fcst = results_ARIMA.forecast(24) # 24시간을 예측합니다.

    a = pd.DataFrame() # a라는 데이터프레임에 예측값을 정리합니다.
    
    for i in range(24):
        a['X2018_7_1_'+str(i+1)+'h']=[fcst[0][i]] # column명을 submission 형태에 맞게 지정합니다.

        
    # 일별 예측
    model = ARIMA(temp_1day['target'], order=get_optimal_params(temp_1day['target'])) # AIC를 최소화하는 최적의 파라미터로 모델링합니다.
    results_ARIMA = model.fit(disp=-1)
    fcst = results_ARIMA.forecast(10) # 10일을 예측합니다.

    for i in range(10):
        a['X2018_7_'+str(i+1)+'_d']=[fcst[0][i]] # column명을 submission 형태에 맞게 지정합니다.
    
    
    # 월별 예측
    # 일별로 예측하여 7월 ~ 11월의 일 수에 맞게 나누어 합산합니다.
    fcst = results_ARIMA.forecast(153)
    a['X2018_7_m'] = [np.sum(fcst[0][:31])] # 7월 
    a['X2018_8_m'] = [np.sum(fcst[0][31:62])] # 8월
    a['X2018_9_m'] = [np.sum(fcst[0][62:92])] # 9월
    a['X2018_10_m'] = [np.sum(fcst[0][92:123])] # 10월
    a['X2018_11_m'] = [np.sum(fcst[0][123:153])] # 11월
    
    a['meter_id'] = key 
    agg[key] = a[submission.columns.tolist()]
    print(key)
print('---- Modeling Done ----')

X692
X1272
X553
X1299
X598
X1003


C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X1010
X1216
X1047
X381
X466
X1420


C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X686
X711
X1251
X828
X1350
X1066


...

C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X755
X925
X1109
X651
X504
X1278


C:\Anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


X1134
X808
X21
X1298


ValueError: min() arg is an empty sequence

In [24]:
output1 = pd.concat(agg, ignore_index=False)
output2 = output1.reset_index().drop(['level_0','level_1'], axis=1)
output2['id'] = output2['meter_id'].str.replace('X','').astype(int)
output2 =  output2.sort_values(by='id', ascending=True).drop(['id'], axis=1).reset_index(drop=True)
output2.to_csv('./SUHYUN/sub_baseline.csv', index=False)

In [25]:
output2.head()

,meter_id,X2018_7_1_1h,X2018_7_1_2h,X2018_7_1_3h,X2018_7_1_4h,X2018_7_1_5h,X2018_7_1_6h,X2018_7_1_7h,X2018_7_1_8h,X2018_7_1_9h,...,X2018_7_6_d,X2018_7_7_d,X2018_7_8_d,X2018_7_9_d,X2018_7_10_d,X2018_7_m,X2018_8_m,X2018_9_m,X2018_10_m,X2018_11_m
0,X2,0.901058,1.069870,1.203352,1.308897,1.392353,1.458343,1.510522,1.551780,1.584403,...,58.142752,58.563014,58.983277,59.403540,59.823803,1932.706751,2336.579243,2645.746124,3131.296081,3414.826936
1,X8,0.000225,0.000225,0.000225,0.000225,0.000225,0.000225,0.000225,0.000225,0.000225,...,0.005278,0.005278,0.005278,0.005278,0.005278,0.163611,0.163611,0.158333,0.163611,0.158333
2,X9,0.210877,0.146053,0.103223,0.074921,0.056213,0.043843,0.035658,0.030238,0.026643,...,0.301531,0.287763,0.273994,0.260226,0.246457,5.079222,-8.152329,-20.487549,-34.188606,-45.683946
3,X13,0.010546,0.008725,0.007336,0.006276,0.005466,0.004848,0.004377,0.004017,0.003742,...,0.058800,0.058800,0.058800,0.058800,0.058800,1.822800,1.822800,1.764000,1.822800,1.764000
4,X14,0.470240,0.463562,0.458882,0.455603,0.453306,0.451696,0.450568,0.449777,0.449224,...,13.092867,13.135800,13.188322,13.244921,13.303254,427.093130,480.749937,519.789644,593.481995,628.885184
